# Cleaning unique user data

In [5]:
# importing necessary packages
import pandas as pd
import requests
from dotenv import load_dotenv
load_dotenv()
import os
import json
import flickrapi
import time
import numpy as np
import re
import pycountry
from clean_flickr_userdata_function_britta import clean_user_data

In [ ]:
# # read csv into df
# df_user_data = pd.read_csv('data/unique_owner.csv')

In [2]:
# read csv into df
df_user_data = pd.read_csv('data/update_user.csv')

In [4]:
df_user_data.shape

(15111, 32)

In [3]:
# check df
df_user_data.head(10)

,Unnamed: 0,stat,person_id,person_nsid,person_ispro,person_is_deleted,person_iconserver,person_iconfarm,person_path_alias,person_has_stats,...,person_photos_firstdate__content,person_photos_count__content,person_has_adfree,person_has_free_standard_shipping,person_has_free_educational_resources,person_mbox_sha1sum__content,person_pro_badge,person_expire,code,message
0,0,ok,92760508@N03,92760508@N03,0.0,0.0,7423.0,8.0,bystuart,0.0,...,1.454308e+09,913.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,1,ok,106185825@N06,106185825@N06,0.0,0.0,2849.0,3.0,NaN,0.0,...,1.382374e+09,1484.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,2,ok,29761931@N02,29761931@N02,0.0,0.0,3117.0,4.0,bonolux,0.0,...,1.219331e+09,505.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,3,ok,196446653@N04,196446653@N04,0.0,0.0,0.0,0.0,fromthepalebluedot,0.0,...,1.662412e+09,23.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,4,ok,67572780@N04,67572780@N04,0.0,0.0,5678.0,6.0,photo-videodavidegasparetto,0.0,...,1.369136e+09,1171.0,0.0,0.0,0.0,1208c33d5eaa609d98f0c530ff9c2376d3d1b8f4,NaN,NaN,NaN,NaN
5,5,ok,124250374@N03,124250374@N03,0.0,0.0,3865.0,4.0,NaN,0.0,...,1.401489e+09,6.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
6,6,ok,28198184@N04,28198184@N04,0.0,0.0,2833.0,3.0,yichenlinsf,0.0,...,1.214954e+09,32683.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
7,7,ok,126764883@N02,126764883@N02,1.0,0.0,3843.0,4.0,fullhdsports,0.0,...,1.407673e+09,29135.0,0.0,0.0,0.0,4c6311d02cb0160e47742e9d8b3247879c4b3a4f,legacy,0.0,NaN,NaN
8,8,ok,8337394@N02,8337394@N02,1.0,0.0,4294.0,5.0,markusbaumgartner,0.0,...,1.179405e+09,7307.0,0.0,0.0,0.0,NaN,standard,0.0,NaN,NaN
9,9,ok,65277369@N05,65277369@N05,1.0,0.0,7866.0,8.0,NaN,0.0,...,1.310856e+09,2368.0,0.0,0.0,0.0,NaN,standard,0.0,NaN,NaN


## Drop unnecessary columns

In [ ]:
# check value count for is deleted
print(df_user_data.person_is_deleted.value_counts())
print(df_user_data.person_is_deleted.isna().sum())
# because there is only 1 value (False) and 24 NaN I decided to drop the column

In [ ]:
# list of columns to drop
columns_to_drop = ['Unnamed: 0.6','Unnamed: 0.5','Unnamed: 0.4', 'Unnamed: 0.3',
       'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'stat','person_nsid','person_is_deleted',
       'person_iconserver','person_iconfarm','person_has_stats',
       'person_description__content',
       'person_photosurl__content', 'person_profileurl__content',
       'person_mobileurl__content','person_has_adfree', 'person_has_free_standard_shipping',
       'person_has_free_educational_resources','person_pro_badge','person_mbox_sha1sum__content','message','person_expire', 'code','person_timezone_timezone']

In [ ]:
# check if its the right number
len(columns_to_drop)

In [ ]:
# drop 26 columns
df_user_data_cleaned = df_user_data.drop(columns=columns_to_drop)

In [ ]:
# check columns
df_user_data_cleaned.columns

## Alter column names

In [ ]:
# replace 'person' with user and delete '_content'
df_user_data_cleaned.columns = df_user_data_cleaned.columns.str.replace('person','user').str.replace('__content','')


In [ ]:
df_user_data_cleaned.columns

## delete NaN from user_location, bc it's our most important column

In [ ]:
# clear NaN values from user_locations
df_user_data_cleaned = df_user_data_cleaned.dropna(subset='user_location').reset_index(drop=True)
#df_user_data_cleaned = df_user_data_cleaned.reset_index(drop=True)

In [ ]:
# check how much is still there
df_user_data_cleaned.shape

In [ ]:
df_user_data_cleaned.head()

In [ ]:
len(df_user_data_cleaned)

In [ ]:
# check percentage of leftover data - 37% left data
x = (len(df_user_data_cleaned)/len(df_user_data))*100
print(x)

## alter dtypes

In [ ]:
# check dtypes
df_user_data_cleaned.info()

In [ ]:
df_user_data_cleaned.user_id

In [ ]:
# user_ispro to bool
df_user_data_cleaned.user_ispro=df_user_data_cleaned.user_ispro.astype('bool')

In [ ]:
df_user_data_cleaned[['user_timezone_offset','user_timezone_timezone_id','user_timezone_label']].sample(5)

In [ ]:
# user_location','user_realname','user_username','user_path_alias,... to string except user_id, bc it needs to be original to query the API
df_user_data_cleaned[['user_location','user_realname','user_username','user_path_alias','user_timezone_offset','user_timezone_timezone_id','user_timezone_label']] = df_user_data_cleaned[['user_location','user_realname','user_username','user_path_alias','user_timezone_offset','user_timezone_timezone_id','user_timezone_label']].astype('string')

In [ ]:

df_user_data_cleaned = df_user_data_cleaned.apply(lambda x: x.str.lower() if x.dtype == "string" else x) 

In [ ]:
# user_id to string
df_user_data_cleaned.user_id=df_user_data_cleaned.user_id.astype('string')

In [ ]:
df_user_data_cleaned.sample(19)

In [ ]:
df_user_data_cleaned.info()

In [ ]:
# user_photos_count to int
df_user_data_cleaned.user_photos_count = df_user_data_cleaned.user_photos_count.astype('int')

In [ ]:
df_user_data_cleaned.user_photos_firstdate

In [ ]:
# user_firstdate unix to datetime
df_user_data_cleaned.user_photos_firstdate = pd.to_datetime(df_user_data_cleaned.user_photos_firstdate,unit='s',errors='coerce')

In [ ]:
# check type
type(df_user_data_cleaned.user_photos_firstdatetaken)

In [ ]:
# alter to string
df_user_data_cleaned.user_photos_firstdatetaken = df_user_data_cleaned.user_photos_firstdatetaken.astype('string')

In [ ]:
df_user_data_cleaned.user_photos_firstdatetaken[9:15]

In [ ]:
df_user_data_cleaned.user_photos_firstdatetaken = pd.to_datetime(df_user_data_cleaned.user_photos_firstdatetaken, dayfirst=True,format='%Y-%m-%d %H:%M:%S',errors='coerce')

In [ ]:
# because coerce replaces error values with NaT 
# firstdatetaken is mysql datetime that starts with 1000-00-00 00:00:00 we "lost" 787 values
df_user_data_cleaned.user_photos_firstdatetaken.count()

In [ ]:
df_user_data_cleaned.info()

## check for duplicates  

In [ ]:
df_user_data_cleaned.duplicated().value_counts()

In [ ]:
# we find more duplicates for one column, otherwise it compares the row
df_user_data_cleaned.duplicated(subset='user_id').value_counts()

In [ ]:
# look at duplicates
df_user_data_cleaned[df_user_data_cleaned.duplicated(keep=False)].sort_values(by='user_id')

## keep existing user id's in user_in_df for updates

In [ ]:
# drop duplicates and store existing user ids to a list
user_in_df=[]
df_user_data_cleaned = df_user_data_cleaned.drop_duplicates(subset='user_id')

for i in df_user_data_cleaned.user_id.unique():
    user_in_df.append(i)

In [ ]:
df_user_data_cleaned.shape

In [ ]:
# check if everything was right
print(user_in_df)
print(len(user_in_df))
# check type
type(user_in_df)

In [ ]:
for x in user_in_df:
    if x == 'd78191777@N00':
        print('yo')
    


In [ ]:
df_user_data_cleaned.sample(5)

## split user location into useful columns

In [ ]:
# check format of user_location
df_user_data_cleaned.user_location

In [ ]:
## conclusion timestamp: as I figured out flickr is creating timezone columns with arguments of location, if they fit in at least on 'city'and/or 'country' format
# check how many values for timezone_offset
df_user_data_cleaned.user_timezone_offset.isna().sum()

In [ ]:
# note there is a wrong TZ
df_user_data_cleaned.loc[7152]

In [ ]:
df_temp = df_user_data_cleaned.user_location.str.split(', ',expand=True)

In [ ]:
df_temp

In [ ]:
df_temp.columns

In [ ]:
df_temp.columns = df_temp.columns.astype('string')

In [ ]:
columns_keep_temp = ['0','1','2']

In [ ]:
df_temp = df_temp.loc[:, columns_keep_temp]

In [ ]:
# "rename" columns from range to names
#df_temp.columns=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21']

In [ ]:
# merge new location columns to dataframe
df_user_data_cleaned=pd.merge(df_user_data_cleaned,df_temp,left_index=True, right_index=True)

In [ ]:
df_user_data_cleaned.head()

In [ ]:
# only the first 3 columns seem to have reliable data, so I dropped the rest
#df_user_data_cleaned = df_user_data_cleaned.drop(columns=['3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21'])

In [ ]:
df_user_data_cleaned.columns

In [ ]:
df_user_data_cleaned.shape

In [ ]:
df_user_data_cleaned.head()

In [ ]:
# check if there is a timezone generated with only usa
df_user_data_cleaned.loc[6316]

## figuring out what happens in the location column

In [ ]:
# change dt to list
# split without expand=true
df_split = df_user_data_cleaned.user_location.str.split(', ')
df_split.tolist()

### decided to only keep the first 3 columns as they fit the format

In [ ]:
three_list=[]
for x in df_split:
    if len(x)<=3:
        three_list.append(x)
print(len(three_list))

In [ ]:
rest_list=[]
for x in df_split:
    if len(x)>=4:
        rest_list.append(x)
print(len(rest_list))

In [ ]:
one_list=[]
for x in df_split:
    if len(x)<=1:
        one_list.append(x)
print(len(one_list))

In [ ]:
len(df_user_data_cleaned.user_location)

In [ ]:
# 94% follow the format
(len(rest_list)/len(three_list))*100

In [6]:
df_user_data_cleaned = clean_user_data(df_user_data)

start cleaning, current shape is: (15111, 32)
shape after dropping NaN's is (7456, 10)
shape after basic cleaning is (7456, 10)
shape after dropping duplicates (7456, 10)


In [7]:
df_user_data_cleaned.head()

,user_id,user_ispro,user_username,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2
0,92760508@N03,False,bystuart,"oakland, united states",1981-05-23 00:00:00,2016-02-01 06:33:12,913,pacific time (us & canada); tijuana,-08:00,pst8pdt,oakland,united states,0
1,29761931@N02,False,ralph foreman,"vitoria-gasteiz, españa",2005-06-19 19:32:34,2008-08-21 15:07:06,505,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,vitoria-gasteiz,españa,0
2,67572780@N04,False,"street, life style photographer","oleggio, italia",2008-01-27 15:22:38,2013-05-21 11:30:20,1171,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,oleggio,italia,0
3,28198184@N04,False,slowpoke_tw,taiwan,2002-01-01 00:00:00,2008-07-01 23:20:27,32683,"beijing, chongqing, hong kong, urumqi",+08:00,asia/chongqing,taiwan,0,0
4,126764883@N02,True,thierry mailhes,"istres, france",2009-11-10 16:13:12,2014-08-10 12:10:48,29135,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,istres,france,0


## getting new df's country,city, state

In [8]:
df_world_countries = pd.read_csv('data/world.csv')

In [ ]:
df_world_countries.head(2)

In [9]:
df_world_countries.drop(columns='id')

,alpha2,alpha3,ar,bg,cs,da,de,el,en,eo,...,ro,ru,sk,sl,sr,sv,th,uk,zh,zh-tw
0,af,afg,أفغانستان,Афганистан,Afghánistán,Afghanistan,Afghanistan,Αφγανιστάν,Afghanistan,Afganio,...,Afganistan,Афганистан,Afganistan,Afganistan,Avganistan,Afghanistan,อัฟกานิสถาน,Афганістан,阿富汗,阿富汗
1,ax,ala,جزر أولاند,Оландски острови,Alandy,Ålandsøerne,Åland,Ώλαντ,Åland Islands,Alando,...,Insulele Åland,Аландские острова,Alandy,Ålandski otoki,Olandska Ostrva,Åland,หมู่เกาะโอลันด์,Аландські острови,奥兰,奧蘭
2,al,alb,ألبانيا,Албания,Albánie,Albanien,Albanien,Αλβανία,Albania,Albanio,...,Albania,Албания,Albánsko,Albanija,Albanija,Albanien,แอลเบเนีย,Албанія,阿尔巴尼亚,阿爾巴尼亞
3,dz,dza,الجزائر,Алжир,Alžírsko,Algeriet,Algerien,Αλγερία,Algeria,Alĝerio,...,Algeria,Алжир,Alžírsko,Alžirija,Alžir,Algeriet,แอลจีเรีย,Алжир,阿尔及利亚,阿爾及利亞
4,as,asm,ساموا الأمريكية,Американска Самоа,Americká Samoa,Amerikansk Samoa,Amerikanisch-Samoa,Αμερικανική Σαμόα,American Samoa,Usona Samoo,...,Samoa americană,Американское Самоа,Americká Samoa,Ameriška Samoa,Američka Samoa,Amerikanska Samoa,อเมริกันซามัว,Американське Самоа,美属萨摩亚,美屬薩摩亞
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,wf,wlf,واليس وفوتونا,Уолис и Футуна,Wallis a Futuna,Wallis og Futuna,Wallis und Futuna,Ουαλίς και Φουτουνά,Wallis and Futuna,Valiso kaj Futuno,...,Wallis și Futuna,Уоллис и Футуна,Wallis a Futuna,Wallis in Futuna,Valis i Futuna,Wallis- och Futunaöarna,วอลิสและฟูตูนา,Волліс і Футуна,瓦利斯和富图纳,瓦利斯和富圖那
245,eh,esh,الصحراء الغربية,Западна Сахара,Západní Sahara,Vestsahara,Westsahara,Δυτική Σαχάρα,Western Sahara,Okcidenta Saharo,...,Sahara Occidentală,САДР,Západná Sahara,Zahodna Sahara,Zapadna Sahara,Västsahara,เวสเทิร์นสะฮารา,Західна Сахара,西撒哈拉,西撒哈拉
246,ye,yem,اليمن,Йемен,Jemen,Yemen,Jemen,Υεμένη,Yemen,Jemeno,...,Yemen,Йемен,Jemen,Jemen,Jemen,Jemen,เยเมน,Ємен,也门,葉門
247,zm,zmb,زامبيا,Замбия,Zambie,Zambia,Sambia,Ζάμπια,Zambia,Zambio,...,Zambia,Замбия,Zambia,Zambija,Zambija,Zambia,แซมเบีย,Замбія,赞比亚,尚比亞


In [ ]:
df_world_countries.columns

In [10]:
#creating a second column en
df_world_countries['en2']= df_world_countries['en']

In [ ]:
# getting countries for laguages flickr has on the website (en,de,es,fr,zh,ko,it,pt,ru)

# creating dictionaries for each language

In [11]:
dict_country_ar= df_world_countries.set_index('ar')['en'].to_dict()
dict_country_bg= df_world_countries.set_index('bg')['en'].to_dict()
dict_country_cs= df_world_countries.set_index('cs')['en'].to_dict()
dict_country_da= df_world_countries.set_index('da')['en'].to_dict()
dict_country_de= df_world_countries.set_index('de')['en'].to_dict()
dict_country_el= df_world_countries.set_index('el')['en'].to_dict()
dict_country_en= df_world_countries.set_index('en')['en2'].to_dict()
dict_country_eo= df_world_countries.set_index('eo')['en'].to_dict()
dict_country_es= df_world_countries.set_index('es')['en'].to_dict()
dict_country_et= df_world_countries.set_index('et')['en'].to_dict()
dict_country_eu= df_world_countries.set_index('eu')['en'].to_dict()
dict_country_fi= df_world_countries.set_index('fi')['en'].to_dict()
dict_country_fr= df_world_countries.set_index('fr')['en'].to_dict()
dict_country_hr= df_world_countries.set_index('hr')['en'].to_dict()
dict_country_hu= df_world_countries.set_index('hu')['en'].to_dict()
dict_country_hy= df_world_countries.set_index('hy')['en'].to_dict()
dict_country_it= df_world_countries.set_index('it')['en'].to_dict()
dict_country_ja= df_world_countries.set_index('ja')['en'].to_dict()
dict_country_ko= df_world_countries.set_index('ko')['en'].to_dict()
dict_country_lt= df_world_countries.set_index('lt')['en'].to_dict()
dict_country_nl= df_world_countries.set_index('nl')['en'].to_dict()
dict_country_no= df_world_countries.set_index('no')['en'].to_dict()
dict_country_pl= df_world_countries.set_index('pl')['en'].to_dict()
dict_country_pt= df_world_countries.set_index('pt')['en'].to_dict()
dict_country_ro= df_world_countries.set_index('ro')['en'].to_dict()
dict_country_ru= df_world_countries.set_index('ru')['en'].to_dict()
dict_country_sk= df_world_countries.set_index('sk')['en'].to_dict()
dict_country_sl= df_world_countries.set_index('sl')['en'].to_dict()
dict_country_sr= df_world_countries.set_index('sr')['en'].to_dict()
dict_country_sv= df_world_countries.set_index('sv')['en'].to_dict()
dict_country_th= df_world_countries.set_index('th')['en'].to_dict()
dict_country_uk= df_world_countries.set_index('uk')['en'].to_dict()
dict_country_zh= df_world_countries.set_index('zh')['en'].to_dict()
dict_country_zh_tw= df_world_countries.set_index('zh-tw')['en'].to_dict()
dict_country_alpha2= df_world_countries.set_index('alpha2')['en'].to_dict()
dict_country_alpha3= df_world_countries.set_index('alpha3')['en'].to_dict()


In [12]:
# add all dictionaries to a list
list_world_countries = [dict_country_de,dict_country_ar,dict_country_bg,dict_country_cs,dict_country_da,dict_country_el,dict_country_en,dict_country_eo,dict_country_es,dict_country_et,dict_country_eu,dict_country_fi,dict_country_fr,dict_country_hr,dict_country_hu,dict_country_hy,dict_country_ja,dict_country_lt,dict_country_nl,dict_country_no,dict_country_pl,dict_country_pt,dict_country_ro,dict_country_sk,dict_country_sl,dict_country_sr,dict_country_sv,dict_country_th,dict_country_uk,dict_country_zh_tw,dict_country_zh,dict_country_ko,dict_country_it,dict_country_ru,dict_country_alpha2,dict_country_alpha3]
len(list_world_countries)

36

In [13]:
dict_country_code = df_world_countries.set_index('en')['alpha2'].to_dict()

In [14]:
dict_country_code = {k.lower(): v.lower() for k, v in dict_country_code.items()}

In [15]:
dict_country_code ['england'] = 'gb'
dict_country_code ['scotland'] = 'gb'
dict_country_code ['united states'] = 'us'
dict_country_code ['uk'] = 'gb'
dict_country_code ['wales'] = 'gb'
dict_country_code ['united kingdom'] = 'gb'
dict_country_code ['ireland'] = 'gb'
dict_country_code ['northern ireland'] = 'gb'

In [ ]:
# country_code_dict_json = json.dumps(dict_country_code,indent=1)
# with open("data/country_code_en_dict.json", "w") as outfile:
#     outfile.write(country_code_dict_json)

In [ ]:
import csv

In [ ]:
with open('data/country_code_en_dict.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in dict_country_code.items():
       writer.writerow([key, value])

In [16]:
df_world_countries[df_world_countries.alpha2 == 'cz']

,id,alpha2,alpha3,ar,bg,cs,da,de,el,en,...,ru,sk,sl,sr,sv,th,uk,zh,zh-tw,en2
59,203,cz,cze,جمهورية التشيك,Чехия,Česko,Tjekkiet,Tschechien,Τσεχία,Czechia,...,Чехия,Česko,Češka,Češka,Tjeckien,เช็กเกีย,Чехія,捷克,捷克,Czechia


In [17]:
list_world_countries


[{'Afghanistan': 'Afghanistan',
  'Åland': 'Åland Islands',
  'Albanien': 'Albania',
  'Algerien': 'Algeria',
  'Amerikanisch-Samoa': 'American Samoa',
  'Andorra': 'Andorra',
  'Angola': 'Angola',
  'Anguilla': 'Anguilla',
  'Antarktis (Sonderstatus durch Antarktisvertrag)': 'Antarctica',
  'Antigua und Barbuda': 'Antigua and Barbuda',
  'Argentinien': 'Argentina',
  'Armenien': 'Armenia',
  'Aruba': 'Aruba',
  'Australien': 'Australia',
  'Österreich': 'Austria',
  'Aserbaidschan': 'Azerbaijan',
  'Bahamas': 'Bahamas',
  'Bahrain': 'Bahrain',
  'Bangladesch': 'Bangladesh',
  'Barbados': 'Barbados',
  'Belarus': 'Belarus',
  'Belgien': 'Belgium',
  'Belize': 'Belize',
  'Benin': 'Benin',
  'Bermuda': 'Bermuda',
  'Bhutan': 'Bhutan',
  'Bolivien': 'Bolivia (Plurinational State of)',
  'Bonaire, Saba, Sint Eustatius': 'Bonaire, Sint Eustatius and Saba',
  'Bosnien und Herzegowina': 'Bosnia and Herzegovina',
  'Botswana': 'Botswana',
  'Bouvetinsel': 'Bouvet Island',
  'Brasilien': 'Braz

In [18]:
df_user_data_cleaned.sample(5)

,user_id,user_ispro,user_username,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2
3871,142537806@N02,True,brad simpson,leicester,2015-10-18 13:50:47,2018-06-17 07:37:58,2114,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,leicester,0,0
4135,87227356@N06,False,tdwrsa,"pretoria, south africa",2003-03-15 16:38:27,2013-08-22 12:19:30,1000,pacific time (us & canada); tijuana,-08:00,pst8pdt,pretoria,south africa,0
2944,9312571@N05,False,montnoirat,vienna; austria,2006-08-12 15:01:39,2007-06-26 08:02:43,1326,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,vienna; austria,0,0
6369,66911745@N00,True,jrmannn,"las vegas, us",NaT,2006-01-23 03:33:08,38242,arizona,-07:00,us/arizona,las vegas,us,0
6836,126948036@N02,True,tompellettdesigner,"pendleton, sc",2011-02-18 10:35:22,2014-09-11 21:59:29,4253,central time (us & canada),-06:00,cst6cdt,pendleton,sc,0


In [ ]:
#replacing NaN with zero, bc Na is the country code for Namibia
df_user_data_cleaned['0'] = df_user_data_cleaned['0'].replace(np.nan,'0')
df_user_data_cleaned['1'] = df_user_data_cleaned['1'].replace(np.nan,'0')
df_user_data_cleaned['2'] = df_user_data_cleaned['2'].replace(np.nan,'0')

In [19]:
# combining the list of dict to one dictionary
combined_dict = {key:value for d in list_world_countries for key, value in d.items()}

In [ ]:
type(combined_dict)

In [20]:
# lower keys and values with dict comprehension
combined_dict = {k.lower(): v.lower() for k, v in combined_dict.items()}


In [21]:
combined_dict

{'afghanistan': 'afghanistan',
 'åland': 'åland islands',
 'albanien': 'albania',
 'algerien': 'algeria',
 'amerikanisch-samoa': 'american samoa',
 'andorra': 'andorra',
 'angola': 'angola',
 'anguilla': 'anguilla',
 'antarktis (sonderstatus durch antarktisvertrag)': 'antarctica',
 'antigua und barbuda': 'antigua and barbuda',
 'argentinien': 'argentina',
 'armenien': 'armenia',
 'aruba': 'aruba',
 'australien': 'australia',
 'österreich': 'austria',
 'aserbaidschan': 'azerbaijan',
 'bahamas': 'bahamas',
 'bahrain': 'bahrain',
 'bangladesch': 'bangladesh',
 'barbados': 'barbados',
 'belarus': 'belarus',
 'belgien': 'belgium',
 'belize': 'belize',
 'benin': 'benin',
 'bermuda': 'bermuda',
 'bhutan': 'bhutan',
 'bolivien': 'bolivia (plurinational state of)',
 'bonaire, saba, sint eustatius': 'bonaire, sint eustatius and saba',
 'bosnien und herzegowina': 'bosnia and herzegovina',
 'botswana': 'botswana',
 'bouvetinsel': 'bouvet island',
 'brasilien': 'brazil',
 'britisches territorium im

In [22]:
# add edge cases to dict
combined_dict ['england'] = 'england'
combined_dict ['scotland'] = 'scotland'
combined_dict ['united states'] = 'united states of america'
combined_dict ['kingdom of cambodia'] = 'cambodia'
combined_dict ['uk'] = 'united kingdom'
combined_dict ['wales'] = 'wales'
combined_dict ['united kingdom'] = 'united kingdom'
combined_dict ['czech republic'] = 'czechia'

In [23]:
# clean columns to find more matching values
df_user_data_cleaned['0'] = df_user_data_cleaned['0'].str.lower().str.strip()
df_user_data_cleaned['1'] = df_user_data_cleaned['1'].str.lower().str.strip()
df_user_data_cleaned['2'] = df_user_data_cleaned['2'].str.lower().str.strip()

In [24]:
# creating a new column empty column with NaN's
df_user_data_cleaned['user_country']= np.nan
df_user_data_cleaned['user_country_code'] = np.nan


In [25]:
df_user_data_cleaned.sample(5)

,user_id,user_ispro,user_username,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2,user_country,user_country_code
882,48907587@N04,False,anthonyr2010,"crawley, uk",1916-01-01 00:00:00,2010-04-02 08:39:58,3236,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,crawley,uk,0,NaN,NaN
360,124244860@N05,True,hobbyphoto18,france,1980-01-01 00:01:28,2014-05-31 12:12:39,4035,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,france,0,0,NaN,NaN
2501,47006668@N02,False,sdfourd,"new brighton, minnesota",2004-10-20 00:00:00,2010-01-29 00:15:11,364,central time (us & canada),-06:00,cst6cdt,new brighton,minnesota,0,NaN,NaN
5457,98212660@N00,False,samory sundjata,"rio de janeiro, brasil",2006-11-25 09:35:56,2010-12-21 23:47:30,427,pacific time (us & canada); tijuana,-08:00,pst8pdt,rio de janeiro,brasil,0,NaN,NaN
4537,11762721@N04,True,arni j.m.,"london, uk",2007-05-17 22:48:39,2008-03-20 21:25:42,850,"gmt: dublin, edinburgh, lisbon, london",+00:00,gmt,london,uk,0,NaN,NaN


## iterating vice versa, bc otherwise shortcuts for states will be recognized as countries

In [26]:
# iterating over every row and dict to find fitting countries
for index, row in df_user_data_cleaned.iterrows():
    zero = row['2']
    if pd.isna(row['user_country']):
        if zero in combined_dict:
            df_user_data_cleaned.at[index, 'user_country']= combined_dict[zero] #fuzzywuzzy .apply

In [27]:
# iterating over every row and dict to find fitting countries
for index, row in df_user_data_cleaned.iterrows():
    zero = row['1']
    if pd.isna(row['user_country']):
        if zero in combined_dict:
            df_user_data_cleaned.at[index, 'user_country']= combined_dict[zero] #fuzzywuzzy .apply

In [28]:
# iterating over every row and dict to find fitting countries
for index, row in df_user_data_cleaned.iterrows():
    zero = row['0']
    if pd.isna(row['user_country']):
        if zero in combined_dict:
            df_user_data_cleaned.at[index, 'user_country']= combined_dict[zero] #fuzzywuzzy .apply


In [29]:
# looping over countries to get alpha 2 in a new column
for index, row in df_user_data_cleaned.iterrows():
    zero = row['user_country']
    if pd.isna(row['user_country_code']):
        if zero in dict_country_code:
            df_user_data_cleaned.at[index, 'user_country_code']= dict_country_code[zero]

In [30]:
dict_country_code

{'afghanistan': 'af',
 'åland islands': 'ax',
 'albania': 'al',
 'algeria': 'dz',
 'american samoa': 'as',
 'andorra': 'ad',
 'angola': 'ao',
 'anguilla': 'ai',
 'antarctica': 'aq',
 'antigua and barbuda': 'ag',
 'argentina': 'ar',
 'armenia': 'am',
 'aruba': 'aw',
 'australia': 'au',
 'austria': 'at',
 'azerbaijan': 'az',
 'bahamas': 'bs',
 'bahrain': 'bh',
 'bangladesh': 'bd',
 'barbados': 'bb',
 'belarus': 'by',
 'belgium': 'be',
 'belize': 'bz',
 'benin': 'bj',
 'bermuda': 'bm',
 'bhutan': 'bt',
 'bolivia (plurinational state of)': 'bo',
 'bonaire, sint eustatius and saba': 'bq',
 'bosnia and herzegovina': 'ba',
 'botswana': 'bw',
 'bouvet island': 'bv',
 'brazil': 'br',
 'british indian ocean territory': 'io',
 'brunei darussalam': 'bn',
 'bulgaria': 'bg',
 'burkina faso': 'bf',
 'burundi': 'bi',
 'cambodia': 'kh',
 'cameroon': 'cm',
 'canada': 'ca',
 'cabo verde': 'cv',
 'cayman islands': 'ky',
 'central african republic': 'cf',
 'chad': 'td',
 'chile': 'cl',
 'china': 'cn',
 'ch

In [31]:
df_user_data_cleaned.head(25)

,user_id,user_ispro,user_username,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2,user_country,user_country_code
0,92760508@N03,False,bystuart,"oakland, united states",1981-05-23 00:00:00,2016-02-01 06:33:12,913,pacific time (us & canada); tijuana,-08:00,pst8pdt,oakland,united states,0,united states of america,us
1,29761931@N02,False,ralph foreman,"vitoria-gasteiz, españa",2005-06-19 19:32:34,2008-08-21 15:07:06,505,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,vitoria-gasteiz,españa,0,spain,es
2,67572780@N04,False,"street, life style photographer","oleggio, italia",2008-01-27 15:22:38,2013-05-21 11:30:20,1171,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,oleggio,italia,0,italy,it
3,28198184@N04,False,slowpoke_tw,taiwan,2002-01-01 00:00:00,2008-07-01 23:20:27,32683,"beijing, chongqing, hong kong, urumqi",+08:00,asia/chongqing,taiwan,0,0,"taiwan, province of china",tw
4,126764883@N02,True,thierry mailhes,"istres, france",2009-11-10 16:13:12,2014-08-10 12:10:48,29135,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,istres,france,0,france,fr
5,8337394@N02,True,markusbaumgartner,switzerland,2007-03-22 14:41:56,2007-05-17 12:27:45,7307,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,switzerland,0,0,switzerland,ch
6,65277369@N05,True,the645zeroquaranta,"firenze, italia",2003-05-14 09:14:10,2011-07-16 22:34:25,2368,<NA>,<NA>,<NA>,firenze,italia,0,italy,it
7,38987545@N05,True,https://fourcorners.photography,"4 corners, us",2006-09-09 07:06:00,2009-06-09 22:45:50,172,central time (us & canada),-06:00,cst6cdt,4 corners,us,0,united states of america,us
8,52957932@N00,False,poperotico,"guarulhos, brasil",1982-12-01 00:00:00,2007-03-27 01:34:49,1539,brasilia,-03:00,america/sao_paulo,guarulhos,brasil,0,brazil,br
9,12818386@N00,True,petyr.rahl,"san diego, usa",2001-06-28 12:00:00,2001-06-28 11:00:00,18116,pacific time (us & canada); tijuana,-08:00,pst8pdt,san diego,usa,0,united states of america,us


In [32]:
combined_dict['kingdom of cambodia']

'cambodia'

### fuzzywuzzy function

In [ ]:
# def fuzzy_match_country(text, choices):
#     # Use fuzzywuzzy to find the best match
#     best_match, score = process.extractOne(text, choices, scorer=fuzz.ratio)
#     # Set a threshold for a minimum score to consider a match
#     threshold = 80  # You can adjust this threshold as needed
#     if score >= threshold:
#         return best_match
#     else:
#         return None

In [ ]:
# Clean up the 'country_code_name' column
#df_user_data_cleaned['user_country_2'] = df_user_data_cleaned['1'].apply(lambda x: fuzzy_match_country(x, combined_dict.keys()))

In [ ]:
#df_user_data_cleaned['user_country_2'].value_counts()

In [ ]:
df_user_data_cleaned.sample(50)

### check edge cases

In [ ]:
df_user_data_cleaned.loc[12714]

In [ ]:
df_user_data_cleaned.loc[12247]

In [ ]:
df_user_data_cleaned.loc[657]

In [ ]:
combined_dict['cz']

In [ ]:
# changing czechia to czech republic ???
#df_user_data_cleaned.user_country.replace('czechia','czech republic', inplace=True)

In [ ]:
# df_user_data_cleaned['user_country_5'] = df_user_data_cleaned['1'].apply(lambda x: fuzzy_match_country(x, countries_dict_na_na.keys()))

In [ ]:
# storing dict to json for function usage
country_dict_json = json.dumps(combined_dict,indent=1)
with open("data/country_dict_all_lang.json", "w") as outfile:
    outfile.write(country_dict_json)

## cities

In [33]:
df_world_cities = pd.read_csv('data/geonames-all-cities-with-a-population-1000.csv', delimiter=';')

In [34]:
df_world_cities.sample(8)

,Geoname ID,Name,ASCII Name,Alternate Names,Feature Class,Feature Code,Country Code,Country name EN,Country Code 2,Admin1 Code,Admin2 Code,Admin3 Code,Admin4 Code,Population,Elevation,DIgital Elevation Model,Timezone,Modification date,LABEL EN,Coordinates
72967,2792985,Léglise,Leglise,"Leglijhe,Leglise,Legliz,Léglise,lai ge li ci,l...",P,PPL,BE,Belgium,NaN,WAL,WLX,84,84033,3865,NaN,395,Europe/Brussels,2012-01-18,Belgium,"49.79985, 5.53652"
101096,670334,Podenii Vechi,Podenii Vechi,"Podeni Vechi,Podenii Vechi",P,PPL,RO,Romania,NaN,30,132271,NaN,NaN,1881,NaN,248,Europe/Bucharest,2013-04-21,Romania,"45.08939, 26.11584"
85379,2788534,Pulle,Pulle,Zandhoven (Pulle),P,PPL,BE,Belgium,NaN,VLG,VAN,11,11054,2810,NaN,9,Europe/Brussels,2020-04-05,Belgium,"51.20317, 4.71434"
110266,8349040,Cockburn Central,Cockburn Central,NaN,P,PPLX,AU,Australia,NaN,08,51820,NaN,NaN,1209,NaN,24,Australia/Perth,2019-07-18,Australia,"-32.12133, 115.84776"
124998,3985129,San Pedro,San Pedro,San Pedro,P,PPL,MX,Mexico,NaN,07,017,NaN,NaN,41165,NaN,1126,America/Monterrey,2018-04-06,Mexico,"25.43333, -103.21667"
37421,5800317,Lake Stevens,Lake Stevens,"Lake Stevens,Lejk Stivens,Lejk Stivuns,Stevens...",P,PPL,US,United States,NaN,WA,061,NaN,NaN,30886,66.0,67,America/Los_Angeles,2017-03-09,United States,"48.0151, -122.06374"
64160,8858092,San Martín Azcatepec,San Martin Azcatepec,"San Martin Azcatepec,San Martín Azcatepec",P,PPL,MX,Mexico,NaN,15,081,NaN,NaN,35390,NaN,2308,America/Mexico_City,2014-04-11,Mexico,"19.68667, -98.97278"
23981,2653901,Camrose,Camrose,NaN,P,PPLA3,GB,United Kingdom,NaN,WLS,Y7,00NS008,NaN,0,NaN,48,Europe/London,2011-07-31,United Kingdom,"51.84139, -5.01"


In [35]:
# clean
df_world_cities.columns = df_world_cities.columns.str.lower().str.replace(' ', '_')


In [36]:
city_columns_to_keep = ['name','ascii_name', 'alternate_names']

In [37]:
df_world_cities_split = df_world_cities.loc[:,city_columns_to_keep]

In [38]:
df_world_cities_split

,name,ascii_name,alternate_names
0,Lyudinovo,Lyudinovo,"Ljudinovo,Lyudinovo,Людиново"
1,Lubyany,Lubyany,"Lubjany,Lubyany,Лубяны"
2,Lobanovo,Lobanovo,"Lobanovo,Лобаново"
3,Linda,Linda,"Linda,Линда"
4,Leskolovo,Leskolovo,"Leskolovo,Lieskula,Лесколово"
...,...,...,...
141116,Limeira,Limeira,"Limeira,Limejra,QGB,li mei la,li mey ra,limaye..."
141117,Itaúna,Itauna,"Itauna,ita'una,yi ta wu na,Итауна,Իտաունա,ইটাউ..."
141118,Itaporanga,Itaporanga,NaN
141119,Itapicuru,Itapicuru,"Missao,Missão"


In [39]:
df_world_cities_split.name = df_world_cities_split.name.str.strip().str.lower()
df_world_cities_split.ascii_name = df_world_cities_split.ascii_name.str.strip().str.lower()
df_world_cities_split.alternate_names = df_world_cities_split.alternate_names.str.strip().str.lower()

In [40]:
df_alternate_names = df_world_cities_split.alternate_names.str.split(',', expand=True)

In [41]:
df_alternate_names.sample(8)

,0,1,2,3,4,5,6,7,8,9,...,259,260,261,262,263,264,265,266,267,268
36521,baduria,badurii,badūria,бадурии,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
131780,shahdad kot,shahdadkot,shahdadpur,shāhdādkot,shāhdādpur,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
23241,salignac,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
15896,takamori,takamori-machi,gao sen,takamori,たかもり,タカモリ,高森,None,None,None,...,None,None,None,None,None,None,None,None,None,None
120997,han-shan-hsien,huanfeng,huan feng,环峰,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
108436,xincheng,xincheng qu,xin cheng,xin cheng qu,新城,新城区,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
32945,cajah's mountain,cajahs mountain,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
34073,isimoe,isimu,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [42]:
df_alternate_names.columns = df_alternate_names.columns.astype('string')

In [43]:
columns_keep = ['0','1','2','3','4','5','6']

In [44]:
df_alternate_names = df_alternate_names.loc[:,columns_keep]

In [45]:
df_alternate_names

,0,1,2,3,4,5,6
0,ljudinovo,lyudinovo,людиново,None,None,None,None
1,lubjany,lubyany,лубяны,None,None,None,None
2,lobanovo,лобаново,None,None,None,None,None
3,linda,линда,None,None,None,None,None
4,leskolovo,lieskula,лесколово,None,None,None,None
...,...,...,...,...,...,...,...
141116,limeira,limejra,qgb,li mei la,li mey ra,limaye'ira,лимеира
141117,itauna,ita'una,yi ta wu na,итауна,իտաունա,ইটাউনা,伊塔乌纳
141118,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141119,missao,missão,None,None,None,None,None


In [ ]:
# merge to split df

In [46]:
df_world_cities_split = pd.merge(df_world_cities_split,df_alternate_names, left_index=True,right_index=True)

In [47]:
df_world_cities_split

,name,ascii_name,alternate_names,0,1,2,3,4,5,6
0,lyudinovo,lyudinovo,"ljudinovo,lyudinovo,людиново",ljudinovo,lyudinovo,людиново,None,None,None,None
1,lubyany,lubyany,"lubjany,lubyany,лубяны",lubjany,lubyany,лубяны,None,None,None,None
2,lobanovo,lobanovo,"lobanovo,лобаново",lobanovo,лобаново,None,None,None,None,None
3,linda,linda,"linda,линда",linda,линда,None,None,None,None,None
4,leskolovo,leskolovo,"leskolovo,lieskula,лесколово",leskolovo,lieskula,лесколово,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
141116,limeira,limeira,"limeira,limejra,qgb,li mei la,li mey ra,limaye...",limeira,limejra,qgb,li mei la,li mey ra,limaye'ira,лимеира
141117,itaúna,itauna,"itauna,ita'una,yi ta wu na,итауна,իտաունա,ইটাউ...",itauna,ita'una,yi ta wu na,итауна,իտաունա,ইটাউনা,伊塔乌纳
141118,itaporanga,itaporanga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141119,itapicuru,itapicuru,"missao,missão",missao,missão,None,None,None,None,None


In [48]:
df_world_cities_split = df_world_cities_split.drop(columns='alternate_names')

In [49]:
df_world_cities_split['ascii_name_2'] = df_world_cities_split.ascii_name

In [50]:
df_world_cities_split

,name,ascii_name,0,1,2,3,4,5,6,ascii_name_2
0,lyudinovo,lyudinovo,ljudinovo,lyudinovo,людиново,None,None,None,None,lyudinovo
1,lubyany,lubyany,lubjany,lubyany,лубяны,None,None,None,None,lubyany
2,lobanovo,lobanovo,lobanovo,лобаново,None,None,None,None,None,lobanovo
3,linda,linda,linda,линда,None,None,None,None,None,linda
4,leskolovo,leskolovo,leskolovo,lieskula,лесколово,None,None,None,None,leskolovo
...,...,...,...,...,...,...,...,...,...,...
141116,limeira,limeira,limeira,limejra,qgb,li mei la,li mey ra,limaye'ira,лимеира,limeira
141117,itaúna,itauna,itauna,ita'una,yi ta wu na,итауна,իտաունա,ইটাউনা,伊塔乌纳,itauna
141118,itaporanga,itaporanga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,itaporanga
141119,itapicuru,itapicuru,missao,missão,None,None,None,None,None,itapicuru


In [51]:
list = ['city of london']
df_world_cities_split[df_world_cities_split.name.isin(list)]

,name,ascii_name,0,1,2,3,4,5,6,ascii_name_2
89442,city of london,city of london,bandaraya london,cathair londan,ceety o lunnon,cidade de londres,cite de londres,city of london,cité de londres,city of london


In [ ]:
#df_world_cities_split.name['london']# = 
#df_world_cities_split.iloc[0,'city of london']

In [52]:
dict_city_ascii_name= df_world_cities_split.set_index('ascii_name_2')['ascii_name'].to_dict()
dict_city_zero= df_world_cities_split.set_index('0')['ascii_name'].to_dict()
dict_city_one= df_world_cities_split.set_index('1')['ascii_name'].to_dict()
dict_city_two= df_world_cities_split.set_index('2')['ascii_name'].to_dict()
dict_city_three= df_world_cities_split.set_index('3')['ascii_name'].to_dict()
dict_city_four= df_world_cities_split.set_index('4')['ascii_name'].to_dict()
dict_city_five= df_world_cities_split.set_index('5')['ascii_name'].to_dict()
dict_city_six= df_world_cities_split.set_index('6')['ascii_name'].to_dict()

In [53]:
dict_city_name= df_world_cities_split.set_index('name')['ascii_name'].to_dict()

In [54]:
list_world_cities = [dict_city_zero,dict_city_ascii_name,dict_city_one,dict_city_two,dict_city_three,dict_city_four,dict_city_five,dict_city_six]

In [55]:
combined_city_dict = {key:value for x in list_world_cities for key,value in x.items() }

In [56]:
combined_city_dict

{'ljudinovo': 'lyudinovo',
 'lubjany': 'lubyany',
 'lobanovo': 'lobanovo',
 'linda': 'lindi',
 'leskolovo': 'leskolovo',
 'koesjva': 'kushva',
 'koerganinsk': 'kurganinsk',
 'krasn.tkach': 'krasnyy tkach',
 "koshekhabl'": "koshekhabl'",
 'korjazhma': 'koryazhma',
 'korobicyno': 'korobitsyno',
 'kondrova': 'kondrovo',
 'kokina': 'kokino',
 'klimov': 'klimovo',
 'kirovskij': 'balpyk bi',
 'kiricy': 'kiritsy',
 'kevsala': 'kevsala',
 'karkazskiy': 'kavkazskiy',
 'karata': 'karata',
 'izhoski': 'iznoski',
 "iban'nkoront": 'ivangorod',
 'isembai': 'ishimbay',
 'inozemcevo': 'inozemtsevo',
 'im.vorovskogo': 'imeni vorovskogo',
 'igra': 'igra',
 'goebacha': 'gubakha',
 'germenchik': 'germenchik',
 'ehtoka': 'etoka',
 'ehlektragorsk': 'elektrogorsk',
 'damadzedava': 'domodedovo',
 'dobroe': 'dobre',
 'dmitrievka': 'dmytrivka',
 'chusavoj': 'chusovoy',
 'chernukha': 'chernukha',
 'chernomorskij': 'chernomorskiy',
 'bazorkino': 'chermen',
 'batalpashinsk': 'cherkessk',
 'cajkovski': 'chaykovskiy

In [57]:
# adding edge cases 
dict_city_name['frankfurt'] = 'frankfurt am main'


In [58]:
df_user_data_cleaned['user_city'] = np.nan

In [59]:
df_user_data_cleaned.sample(20)

,user_id,user_ispro,user_username,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2,user_country,user_country_code,user_city
6877,47512593@N07,False,blue62photography,u.k.,2009-05-24 11:08:21,2010-03-14 09:06:55,3619,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,u.k.,0,0,NaN,NaN,NaN
6766,137314702@N02,False,broadcast journalist dustin murrell,"des moines, iowa, united states",2015-05-01 13:24:26,2016-01-01 19:57:04,237,pacific time (us & canada); tijuana,-08:00,pst8pdt,des moines,iowa,united states,united states of america,us,NaN
3396,64113661@N04,False,l a d y ♔ m o n y,"jeddah, ksa",2011-01-04 05:11:38,2011-06-19 03:02:50,176,"kuwait, riyadh",+03:00,asia/kuwait,jeddah,ksa,0,NaN,NaN,NaN
7435,22465522@N02,False,hartness library,"randolph center, vermont, usa",1850-06-01 15:44:07,2008-01-04 14:49:15,4202,pacific time (us & canada); tijuana,-08:00,pst8pdt,randolph center,vermont,usa,united states of america,us,NaN
3605,35473328@N00,True,dhlake,"indialantic/fl, usa",2002-10-02 04:08:11,2006-09-04 21:18:25,2505,eastern time (us & canada),-05:00,est5edt,indialantic/fl,usa,0,united states of america,us,NaN
5829,20502683@N00,False,roccu1977,"forlì, italy",2003-12-28 15:49:12,2005-11-25 10:40:44,3070,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,forlì,italy,0,italy,it,NaN
2067,84351449@N00,True,yannig van de wouwer,"boom, belgium",2003-01-01 00:00:00,2006-07-31 17:42:09,15750,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,boom,belgium,0,belgium,be,NaN
1283,22872721@N00,True,erwin-edmonton,canada,2005-07-12 21:05:26,2006-11-01 05:27:21,7531,mountain time (us & canada),-07:00,us/mountain,canada,0,0,canada,ca,NaN
1304,61635869@N00,True,vurnman,"nevada city, ca, usa",2003-12-25 14:11:23,2005-10-31 04:56:13,4921,pacific time (us & canada); tijuana,-08:00,pst8pdt,nevada city,ca,usa,united states of america,us,NaN
2950,22131063@N04,True,latourinfernal,"tartane, martinique",2000-01-01 00:01:44,2007-12-24 22:06:57,2801,atlantic time (canada),-04:00,etc/gmt+4,tartane,martinique,0,martinique,mq,NaN


In [ ]:
df_user_data_cleaned.loc[12316]

In [ ]:
# for index, row in df_user_data_cleaned.iterrows():
#     x = row['0']
#     if pd.isna(row['user_city']):
#         if x in combined_city_dict:
#             df_user_data_cleaned.at[index, 'user_city']= combined_city_dict[x]

In [60]:
combined_city_dict['frankfurt']

'frankfort'

In [61]:
dict_city_name['frankfurt']

'frankfurt am main'

## frankfurt causes trouble bc there is an alternative name of frankfort in us
so I decided to run the real name dict prior

In [62]:
for index, row in df_user_data_cleaned.iterrows():
    x = row['0']
    if pd.isna(row['user_city']):
        if x in dict_city_name:
            df_user_data_cleaned.at[index, 'user_city']= dict_city_name[x]

In [63]:
for index, row in df_user_data_cleaned.iterrows():
    x = row['1']
    if pd.isna(row['user_city']):
        if x in dict_city_name:
            df_user_data_cleaned.at[index, 'user_city']= dict_city_name[x]

In [64]:
for index, row in df_user_data_cleaned.iterrows():
    x = row['2']
    if pd.isna(row['user_city']):
        if x in dict_city_name:
            df_user_data_cleaned.at[index, 'user_city']= dict_city_name[x]

In [65]:
df_user_data_cleaned.head(20)

,user_id,user_ispro,user_username,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2,user_country,user_country_code,user_city
0,92760508@N03,False,bystuart,"oakland, united states",1981-05-23 00:00:00,2016-02-01 06:33:12,913,pacific time (us & canada); tijuana,-08:00,pst8pdt,oakland,united states,0,united states of america,us,oakland
1,29761931@N02,False,ralph foreman,"vitoria-gasteiz, españa",2005-06-19 19:32:34,2008-08-21 15:07:06,505,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,vitoria-gasteiz,españa,0,spain,es,espana
2,67572780@N04,False,"street, life style photographer","oleggio, italia",2008-01-27 15:22:38,2013-05-21 11:30:20,1171,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,oleggio,italia,0,italy,it,oleggio
3,28198184@N04,False,slowpoke_tw,taiwan,2002-01-01 00:00:00,2008-07-01 23:20:27,32683,"beijing, chongqing, hong kong, urumqi",+08:00,asia/chongqing,taiwan,0,0,"taiwan, province of china",tw,NaN
4,126764883@N02,True,thierry mailhes,"istres, france",2009-11-10 16:13:12,2014-08-10 12:10:48,29135,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,istres,france,0,france,fr,istres
5,8337394@N02,True,markusbaumgartner,switzerland,2007-03-22 14:41:56,2007-05-17 12:27:45,7307,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,switzerland,0,0,switzerland,ch,NaN
6,65277369@N05,True,the645zeroquaranta,"firenze, italia",2003-05-14 09:14:10,2011-07-16 22:34:25,2368,<NA>,<NA>,<NA>,firenze,italia,0,italy,it,NaN
7,38987545@N05,True,https://fourcorners.photography,"4 corners, us",2006-09-09 07:06:00,2009-06-09 22:45:50,172,central time (us & canada),-06:00,cst6cdt,4 corners,us,0,united states of america,us,us
8,52957932@N00,False,poperotico,"guarulhos, brasil",1982-12-01 00:00:00,2007-03-27 01:34:49,1539,brasilia,-03:00,america/sao_paulo,guarulhos,brasil,0,brazil,br,guarulhos
9,12818386@N00,True,petyr.rahl,"san diego, usa",2001-06-28 12:00:00,2001-06-28 11:00:00,18116,pacific time (us & canada); tijuana,-08:00,pst8pdt,san diego,usa,0,united states of america,us,san diego


In [66]:
df_user_data_cleaned.user_city.isna().sum()

1638

In [67]:
for index, row in df_user_data_cleaned.iterrows():
    x = row['0']
    if pd.isna(row['user_city']):
        if x in combined_city_dict:
            df_user_data_cleaned.at[index, 'user_city']= combined_city_dict[x]

In [68]:
for index, row in df_user_data_cleaned.iterrows():
    x = row['1']
    if pd.isna(row['user_city']):
        if x in combined_city_dict:
            df_user_data_cleaned.at[index, 'user_city']= combined_city_dict[x]

In [69]:
for index, row in df_user_data_cleaned.iterrows():
    x = row['2']
    if pd.isna(row['user_city']):
        if x in combined_city_dict:
            df_user_data_cleaned.at[index, 'user_city']= combined_city_dict[x]

In [70]:
df_user_data_cleaned.sample(15)

,user_id,user_ispro,user_username,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2,user_country,user_country_code,user_city
291,9082612@N05,True,gilles san martin,"namur, belgium",2005-03-26 11:11:25,2007-06-19 22:35:42,3271,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,namur,belgium,0,belgium,be,namur
1844,55216934@N02,False,akafabio1,"bologna, italia",2010-04-18 19:33:36,2010-10-28 20:43:44,367,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,bologna,italia,0,italy,it,bologna
605,89411509@N03,False,tschügge,switzerland,2010-08-01 05:59:22,2013-02-27 10:28:50,249,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,switzerland,0,0,switzerland,ch,NaN
5555,39244790@N04,False,wayne national forest,united states,2009-08-06 11:35:32,2021-11-16 21:07:11,242,eastern time (us & canada),-05:00,est5edt,united states,0,0,united states of america,us,NaN
3648,38942485@N05,False,prozla,"reutlingen, germany",2008-10-19 15:57:26,2010-08-28 19:21:31,464,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,reutlingen,germany,0,germany,de,reutlingen
5317,22983231@N07,False,juanmamateos,"cáceres, españa",2009-04-26 17:21:39,2009-05-12 21:04:47,985,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,cáceres,españa,0,spain,es,caceres
4528,45142629@N03,False,dmaclego,poland,2004-11-27 11:13:51,2009-11-30 13:40:53,492,"sarajevo, skopje, warsaw, zagreb",+01:00,europe/sarajevo,poland,0,0,poland,pl,poland
3426,111772629@N08,False,spmrc13,"villa estense, italia",2013-04-25 17:30:18,2013-12-16 21:36:41,762,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,villa estense,italia,0,italy,it,villa estense
4059,54656712@N06,False,jaimegfoto,"madrid, spain",2010-11-20 18:20:22,2010-11-25 21:30:14,194,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,madrid,spain,0,spain,es,madrid
5790,8558150@N07,True,amy_elizabeth_west,"minneapolis, usa",2003-02-20 05:34:17,2007-06-04 01:40:46,1345,central time (us & canada),-06:00,cst6cdt,minneapolis,usa,0,united states of america,us,minneapolis


In [71]:
df_user_data_cleaned.tail(20)

,user_id,user_ispro,user_username,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2,user_country,user_country_code,user_city
7436,78627699@N00,True,kentlarus,"reykjavk, iceland",2003-07-28 00:37:32,2006-08-27 00:47:18,53224,"casablanca, monrovia",+00:00,africa/casablanca,reykjavk,iceland,0,iceland,is,NaN
7437,85793406@N03,False,σℓγα ƹӝʒ,moldova,2002-01-13 12:06:00,2013-08-19 19:37:59,988,bucharest,+02:00,europe/bucharest,moldova,0,0,"moldova, republic of",md,krutoiarivka
7438,193948571@N04,False,mmb foto,"poznań, poland",2019-01-20 12:27:24,2021-09-17 13:11:06,76,<NA>,<NA>,<NA>,poznań,poland,0,poland,pl,poznan
7439,57966985@N04,False,arabelly.ascoli,"recife, brasil",2008-01-12 18:00:59,2011-01-07 00:25:21,115,brasilia,-03:00,america/sao_paulo,recife,brasil,0,brazil,br,recife
7440,94200309@N02,False,german tenorio,"oaxada de juárez, méxico",2004-01-01 01:17:32,2013-03-26 01:10:55,3326,pacific time (us & canada); tijuana,-08:00,pst8pdt,oaxada de juárez,méxico,0,mexico,mx,mexico
7441,44124366581@N01,False,xavmp,"sunchales, argentina",2000-01-01 00:00:17,2004-06-30 10:28:32,5961,"buenos aires, georgetown",-03:00,america/buenos_aires,sunchales,argentina,0,argentina,ar,sunchales
7442,76704164@N00,False,dice man,"mauá, brasil",2006-07-01 00:00:00,2006-11-20 21:42:14,3549,brasilia,-03:00,america/sao_paulo,mauá,brasil,0,brazil,br,maua
7443,56968592@N03,False,st. francis cainta,"cainta, rizal, philippines",2008-03-16 19:30:19,2010-12-13 06:31:29,49083,"beijing, chongqing, hong kong, urumqi",+08:00,asia/chongqing,cainta,rizal,philippines,philippines,ph,cainta
7444,15034930@N06,False,neil kesterson,"lexington, ky, usa",1987-01-01 00:00:00,2007-12-05 04:06:43,938,eastern time (us & canada),-05:00,est5edt,lexington,ky,usa,united states of america,us,lexington
7445,191630354@N06,False,thinhnh3009,"ho chi minh city, vietnam",2019-07-03 19:49:44,2021-01-01 10:55:47,145,<NA>,<NA>,<NA>,ho chi minh city,vietnam,0,viet nam,vn,ho chi minh city


In [ ]:
## check edge cases

In [ ]:
df_user_data_cleaned.loc[9958]

In [ ]:
df_user_data_cleaned.loc[5935]

In [ ]:
df_user_data_cleaned.loc[8]

In [ ]:
df_user_data_cleaned.shape

In [72]:
df_user_data_cleaned.user_city.isna().sum()

1315

In [73]:
df_user_data_cleaned.user_country.isna().sum()

1152

In [ ]:
df_world_cities.sample(7)

In [74]:
world_city_columns_keep = ['ascii_name','country_name_en','country_code','population','timezone','coordinates']

In [75]:
df_world_cities = df_world_cities.loc[:,world_city_columns_keep]

In [76]:
# add prefix except ascii_name
df_world_cities = df_world_cities.add_prefix('city_')


In [ ]:
#df_world_cities['user_city'] = df_world_cities['city_ascii_name']

In [77]:
df_world_cities.sample(5)

,city_ascii_name,city_country_name_en,city_country_code,city_population,city_timezone,city_coordinates
107626,Sempere,Spain,ES,34,Europe/Madrid,"38.92014, -0.4814"
112764,Sahbuz,Azerbaijan,AZ,3127,Asia/Baku,"39.40722, 45.57389"
121599,Veglio,Italy,IT,660,Europe/Rome,"45.64057, 8.11412"
70041,Naranjito,Honduras,HN,3813,America/Tegucigalpa,"14.95297, -88.68333"
9001,Yuzawa,Japan,JP,44346,Asia/Tokyo,"39.16667, 140.5"


In [ ]:
#df_world_cities[df_world_cities.city_country_name_en=="Korea, Republic of"]

In [ ]:
# df_world_countries.head()
# df_world_countries[df_world_countries.alpha2=='cz']

In [ ]:
#df_world_countries['Czech Republic']='Czechia'

In [ ]:
#df_user_data_cleaned_city = df_user_data_cleaned.merge(df_world_cities, left_on='user_city',right_on='city_ascii_name',how='left')

In [ ]:
#df_user_data_cleaned_country=df_user_data_cleaned.merge(df_world_cities, left_on='user_country',right_on='city_country_name_en',how='left')

In [ ]:
#df_user_data_cleaned_country.head(20)

In [78]:
df_world_cities.city_ascii_name = df_world_cities.city_ascii_name.str.lower()

In [79]:
df_world_cities.city_country_name_en = df_world_cities.city_country_name_en.str.lower()
df_user_data_cleaned.user_country = df_user_data_cleaned.user_country.str.lower()
df_world_cities.city_timezone = df_world_cities.city_timezone.str.lower()
df_world_cities.city_country_code = df_world_cities.city_country_code.str.lower()

In [80]:
df_world_cities = df_world_cities.apply(lambda x: x.astype('string').str.strip() if x.dtype == "object" else x) 

In [81]:
df_world_cities.head()

,city_ascii_name,city_country_name_en,city_country_code,city_population,city_timezone,city_coordinates
0,lyudinovo,russian federation,ru,41392,europe/moscow,"53.86639, 34.44778"
1,lubyany,russian federation,ru,1899,europe/moscow,"56.0378, 51.40001"
2,lobanovo,russian federation,ru,3563,asia/yekaterinburg,"57.8595, 56.3026"
3,linda,russian federation,ru,5489,europe/moscow,"56.61571, 44.09544"
4,leskolovo,russian federation,ru,4039,europe/moscow,"60.26341, 30.45462"


In [ ]:
df_world_cities.info()

In [82]:
# split coordinates and merge to df world cities
df_c_split = df_world_cities.city_coordinates.str.split(', ', expand=True)
df_c_split.columns = df_c_split.columns.astype('string')
df_c_split = df_c_split.rename(columns={'0':'city_lat','1':'city_long'})

In [83]:
df_c_split.head()

,city_lat,city_long
0,53.86639,34.44778
1,56.0378,51.40001
2,57.8595,56.3026
3,56.61571,44.09544
4,60.26341,30.45462


In [84]:
df_world_cities = df_world_cities.merge(df_c_split, left_index=True,right_index=True,how='left')

In [85]:
df_world_cities.head()

,city_ascii_name,city_country_name_en,city_country_code,city_population,city_timezone,city_coordinates,city_lat,city_long
0,lyudinovo,russian federation,ru,41392,europe/moscow,"53.86639, 34.44778",53.86639,34.44778
1,lubyany,russian federation,ru,1899,europe/moscow,"56.0378, 51.40001",56.0378,51.40001
2,lobanovo,russian federation,ru,3563,asia/yekaterinburg,"57.8595, 56.3026",57.8595,56.3026
3,linda,russian federation,ru,5489,europe/moscow,"56.61571, 44.09544",56.61571,44.09544
4,leskolovo,russian federation,ru,4039,europe/moscow,"60.26341, 30.45462",60.26341,30.45462


In [86]:
df_user_data_cleaned_final=df_user_data_cleaned.merge(df_world_cities, left_on=['user_country_code','user_city'],right_on=['city_country_code','city_ascii_name'],how='left')

In [87]:
df_user_data_cleaned_final.head(20)

,user_id,user_ispro,user_username,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,...,user_country_code,user_city,city_ascii_name,city_country_name_en,city_country_code,city_population,city_timezone,city_coordinates,city_lat,city_long
0,92760508@N03,False,bystuart,"oakland, united states",1981-05-23 00:00:00,2016-02-01 06:33:12,913,pacific time (us & canada); tijuana,-08:00,pst8pdt,...,us,oakland,oakland,united states,us,2602.0,america/new_york,"44.54034, -69.72199",44.54034,-69.72199
1,92760508@N03,False,bystuart,"oakland, united states",1981-05-23 00:00:00,2016-02-01 06:33:12,913,pacific time (us & canada); tijuana,-08:00,pst8pdt,...,us,oakland,oakland,united states,us,1875.0,america/new_york,"39.40787, -79.40671",39.40787,-79.40671
2,92760508@N03,False,bystuart,"oakland, united states",1981-05-23 00:00:00,2016-02-01 06:33:12,913,pacific time (us & canada); tijuana,-08:00,pst8pdt,...,us,oakland,oakland,united states,us,1202.0,america/chicago,"41.83583, -96.46697",41.83583,-96.46697
3,92760508@N03,False,bystuart,"oakland, united states",1981-05-23 00:00:00,2016-02-01 06:33:12,913,pacific time (us & canada); tijuana,-08:00,pst8pdt,...,us,oakland,oakland,united states,us,1507.0,america/chicago,"41.30916, -95.39667",41.30916,-95.39667
4,92760508@N03,False,bystuart,"oakland, united states",1981-05-23 00:00:00,2016-02-01 06:33:12,913,pacific time (us & canada); tijuana,-08:00,pst8pdt,...,us,oakland,oakland,united states,us,1073.0,america/chicago,"34.1001, -96.79389",34.1001,-96.79389
5,92760508@N03,False,bystuart,"oakland, united states",1981-05-23 00:00:00,2016-02-01 06:33:12,913,pacific time (us & canada); tijuana,-08:00,pst8pdt,...,us,oakland,oakland,united states,us,13165.0,america/new_york,"41.01315, -74.26431",41.01315,-74.26431
6,92760508@N03,False,bystuart,"oakland, united states",1981-05-23 00:00:00,2016-02-01 06:33:12,913,pacific time (us & canada); tijuana,-08:00,pst8pdt,...,us,oakland,oakland,united states,us,1437.0,america/chicago,"42.58109, -93.44326",42.58109,-93.44326
7,92760508@N03,False,bystuart,"oakland, united states",1981-05-23 00:00:00,2016-02-01 06:33:12,913,pacific time (us & canada); tijuana,-08:00,pst8pdt,...,us,oakland,oakland,united states,us,2829.0,america/new_york,"28.555, -81.63313",28.555,-81.63313
8,92760508@N03,False,bystuart,"oakland, united states",1981-05-23 00:00:00,2016-02-01 06:33:12,913,pacific time (us & canada); tijuana,-08:00,pst8pdt,...,us,oakland,oakland,united states,us,1569.0,america/new_york,"40.98978, -80.36951",40.98978,-80.36951
9,92760508@N03,False,bystuart,"oakland, united states",1981-05-23 00:00:00,2016-02-01 06:33:12,913,pacific time (us & canada); tijuana,-08:00,pst8pdt,...,us,oakland,oakland,united states,us,1232.0,america/new_york,"33.98293, -80.48842",33.98293,-80.48842


In [88]:
df_user_data_cleaned_final.loc[10]

user_id                                              92760508@N03
user_ispro                                                  False
user_username                                            bystuart
user_location                              oakland, united states
user_photos_firstdatetaken                    1981-05-23 00:00:00
user_photos_firstdate                         2016-02-01 06:33:12
user_photos_count                                             913
user_timezone_label           pacific time (us & canada); tijuana
user_timezone_offset                                       -08:00
user_timezone_timezone_id                                 pst8pdt
0                                                         oakland
1                                                   united states
2                                                               0
user_country                             united states of america
user_country_code                                              us
user_city 

In [89]:
df_user_data_cleaned_final[df_user_data_cleaned_final.city_ascii_name=='london']

,user_id,user_ispro,user_username,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,...,user_country_code,user_city,city_ascii_name,city_country_name_en,city_country_code,city_population,city_timezone,city_coordinates,city_lat,city_long
143,41087279@N00,True,photographer695,"london, england",NaT,2008-11-09 11:41:21,483799,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,...,gb,london,london,united kingdom,gb,8961989.0,europe/london,"51.50853, -0.12574",51.50853,-0.12574
205,110597967@N06,False,.willwalker,"london, united kingdom",2004-12-26 13:18:50,2013-12-04 15:09:43,234,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,...,gb,london,london,united kingdom,gb,8961989.0,europe/london,"51.50853, -0.12574",51.50853,-0.12574
231,96232193@N00,False,sharonodea,"london, uk",2002-11-01 04:59:05,2006-04-10 11:03:48,5865,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,...,gb,london,london,united kingdom,gb,8961989.0,europe/london,"51.50853, -0.12574",51.50853,-0.12574
829,13059914@N00,True,mazarak,"london, uk",2002-11-24 16:06:24,2006-08-10 22:10:42,2072,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,...,gb,london,london,united kingdom,gb,8961989.0,europe/london,"51.50853, -0.12574",51.50853,-0.12574
933,78364563@N00,True,estherase,"london, united kingdom",1985-01-01 00:00:00,2005-05-10 19:27:16,11808,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,...,gb,london,london,united kingdom,gb,8961989.0,europe/london,"51.50853, -0.12574",51.50853,-0.12574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10016,89408706@N02,True,benjamin parker photography uk,"london, uk",2003-01-29 15:43:39,2012-11-30 21:00:34,2510,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,...,gb,london,london,united kingdom,gb,8961989.0,europe/london,"51.50853, -0.12574",51.50853,-0.12574
10067,93602439@N02,False,sherpa expeditions,"london, uk",1980-01-01 00:00:03,2013-02-27 11:56:17,863,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,...,gb,london,london,united kingdom,gb,8961989.0,europe/london,"51.50853, -0.12574",51.50853,-0.12574
10180,7717749@N06,False,opopododo,"london, england",1989-08-01 00:00:00,2007-04-07 19:56:29,522,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,...,gb,london,london,united kingdom,gb,8961989.0,europe/london,"51.50853, -0.12574",51.50853,-0.12574
10201,86127290@N00,False,birdsigh,"london, uk",2003-01-01 12:27:08,2006-06-27 22:54:28,2715,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,...,gb,london,london,united kingdom,gb,8961989.0,europe/london,"51.50853, -0.12574",51.50853,-0.12574


In [90]:
df_user_data_cleaned_final.shape

(10402, 24)

In [91]:
df_user_data_cleaned_final.city_coordinates.isna().sum()

3121

In [ ]:
#df_user_data_cleaned_city.shape

In [ ]:
#df_user_data_cleaned_country.shape

In [92]:
df_user_data_cleaned_final.sample(15)

,user_id,user_ispro,user_username,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,...,user_country_code,user_city,city_ascii_name,city_country_name_en,city_country_code,city_population,city_timezone,city_coordinates,city_lat,city_long
6231,61763054@N04,False,rodrigo g. tinoco,"belo horizonte, brasil",2022-07-13 17:34:19,2023-07-10 16:15:17,4,brasilia,-03:00,america/sao_paulo,...,br,belo horizonte,belo horizonte,brazil,br,2721564.0,america/sao_paulo,"-19.92083, -43.93778",-19.92083,-43.93778
9693,56783837@N00,False,snugglebeartx,"carrollton, usa",2006-10-07 13:08:28,2007-11-15 04:39:03,13274,central time (us & canada),-06:00,cst6cdt,...,us,carrollton,carrollton,united states,us,988.0,america/chicago,"33.26169, -88.09503",33.26169,-88.09503
10181,24289706@N00,True,dee mcevoy,ireland,2004-09-02 04:08:13,2006-03-17 16:45:11,2751,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,...,gb,west springfield,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>
6606,193340322@N05,False,bob w griffiths,"shrewsbury, united kingdom",2011-09-22 20:22:28,2021-06-30 12:57:06,348,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,...,gb,shrewsbury,shrewsbury,united kingdom,gb,76782.0,europe/london,"52.71009, -2.75208",52.71009,-2.75208
8559,34853247@N04,False,mr.mellow,"frampton, uk",2003-09-06 12:29:08,2009-01-29 14:59:11,435,"gmt: dublin, edinburgh, lisbon, london",+00:00,europe/london,...,gb,uk,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>
8626,42195642@N00,True,dearth !,"denver, co, usa",2004-02-23 22:10:56,2006-10-11 06:58:07,4327,mountain time (us & canada),-07:00,us/mountain,...,us,denver,denver,united states,us,715522.0,america/denver,"39.73915, -104.9847",39.73915,-104.9847
10184,147329032@N03,False,furkan galleria 3,bursa,2013-07-18 15:12:41,2019-03-06 21:18:20,944,<NA>,<NA>,<NA>,...,NaN,bursa,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>
10069,40566216@N04,True,jamisings,"costa mesa, usa",2005-01-01 12:01:02,2009-07-18 00:45:38,13334,pacific time (us & canada); tijuana,-08:00,pst8pdt,...,us,costa mesa,costa mesa,united states,us,113204.0,america/los_angeles,"33.64113, -117.91867",33.64113,-117.91867
6671,28967300@N03,True,sweetpea646,"wellington, new zealand",1985-01-01 00:00:00,2008-07-29 10:03:56,4772,"auckland, wellington",+12:00,pacific/auckland,...,nz,wellington,wellington,new zealand,nz,381900.0,pacific/auckland,"-41.28664, 174.77557",-41.28664,174.77557
2494,23226161@N08,True,lars_snapshooter,"græsted, danmark",2006-08-27 17:14:25,2008-01-25 17:05:59,3411,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,...,dk,graested,graested,denmark,dk,3561.0,europe/copenhagen,"56.06558, 12.28512",56.06558,12.28512


In [ ]:
# compare countries
#df_user_data_cleaned['equal_country'] = df_user_data_cleaned['user_country'].eq(df_user_data_cleaned['city_country_name_en'])

In [ ]:
df_user_data_cleaned_final.head(30)

In [ ]:
# keep false + NaN
#df_user_data_cleaned[df_user_data_cleaned['equal_country']== True]

In [ ]:
df_user_data_cleaned_final.city_coordinates.count()

In [ ]:
#df_user_data_cleaned_final[df_user_data_cleaned_final[user_country]]

In [ ]:
df_user_data_cleaned_final.shape


In [ ]:
df_user_data_cleaned_final.info()

In [93]:
columns_to_drop_final = ['0','1','2']
df_user_data_cleaned_final = df_user_data_cleaned_final.drop(columns=columns_to_drop_final)

In [94]:
df_user_data_cleaned_final = df_user_data_cleaned_final.drop_duplicates(subset='user_id')

In [95]:
df_user_data_cleaned_final.shape

(7456, 21)

In [96]:
df_user_data_cleaned.head()

,user_id,user_ispro,user_username,user_location,user_photos_firstdatetaken,user_photos_firstdate,user_photos_count,user_timezone_label,user_timezone_offset,user_timezone_timezone_id,0,1,2,user_country,user_country_code,user_city
0,92760508@N03,False,bystuart,"oakland, united states",1981-05-23 00:00:00,2016-02-01 06:33:12,913,pacific time (us & canada); tijuana,-08:00,pst8pdt,oakland,united states,0,united states of america,us,oakland
1,29761931@N02,False,ralph foreman,"vitoria-gasteiz, españa",2005-06-19 19:32:34,2008-08-21 15:07:06,505,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,vitoria-gasteiz,españa,0,spain,es,espana
2,67572780@N04,False,"street, life style photographer","oleggio, italia",2008-01-27 15:22:38,2013-05-21 11:30:20,1171,"amsterdam, berlin, bern, rome, stockholm, vienna",+01:00,europe/amsterdam,oleggio,italia,0,italy,it,oleggio
3,28198184@N04,False,slowpoke_tw,taiwan,2002-01-01 00:00:00,2008-07-01 23:20:27,32683,"beijing, chongqing, hong kong, urumqi",+08:00,asia/chongqing,taiwan,0,0,"taiwan, province of china",tw,NaN
4,126764883@N02,True,thierry mailhes,"istres, france",2009-11-10 16:13:12,2014-08-10 12:10:48,29135,"brussels, copenhagen, madrid, paris",+01:00,europe/brussels,istres,france,0,france,fr,istres


In [ ]:
#df_user_data_cleaned_final[df_user_data_cleaned_final['user_id'=='78191777@N00']]

In [97]:
df_user_data_cleaned_final = df_user_data_cleaned_final.dropna(subset='user_country')

In [98]:
df_user_data_cleaned_final.columns

Index(['user_id', 'user_ispro', 'user_username', 'user_location',
       'user_photos_firstdatetaken', 'user_photos_firstdate',
       'user_photos_count', 'user_timezone_label', 'user_timezone_offset',
       'user_timezone_timezone_id', 'user_country', 'user_country_code',
       'user_city', 'city_ascii_name', 'city_country_name_en',
       'city_country_code', 'city_population', 'city_timezone',
       'city_coordinates', 'city_lat', 'city_long'],
      dtype='object')

In [100]:
df_user_data_cleaned_final = df_user_data_cleaned_final.drop(columns=['user_timezone_label',
       'user_timezone_offset', 'user_timezone_timezone_id','city_country_code','city_country_name_en','city_ascii_name'])

In [101]:
df_user_data_cleaned_final.shape

(6304, 15)

In [102]:
df_user_data_cleaned_final.to_csv('data/clean_user_update.csv', sep=',')

In [103]:
import psycopg2 # needed to get database exception errors when uploading dataframe
import requests
from sql_functions import get_engine

In [104]:
table_name = 'clean_user_update'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This may take some time ...
engine = get_engine()
schema = 'capstone_jorittega'
# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        df_user_data_cleaned_final.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The clean_user_update table was imported successfully.
